In [7]:
from DataConverterOverlay_TEST import *
from IPython.display import clear_output

verbose = True
dacOverlay = DataConverterOverlay("sys_design.bit", download = True)

seeds = [0xBEEF,0xABCD,0x1234,0xFEED,0xBEAD]
seed = seeds[0]

cmd = [""]
cmds = {k:i-i%2 for i,k in enumerate(["choose",'c',"rst",'r',"random",'rs',"trig",'t',"hlt",'h',"plt",'p',"twave",'tw', "burstSize",'bs', "readBurstSize", "rbs", "help", "scale", "s", "simple_test", "st", "pwl_test", "tpw", "draw_pwl", "dpw", "full_test", "ft"])}
cmds['q'] = 0xaaaaaa
removeBlanks = lambda li: [el for el in li if el != '']
helpMsg = "\n\nCommands:\n\
choose n (c n) Choose the nth seed for random generator\n\
rst (r) Resets the system\n\
random (rs) Sends a random sequence of values to the DAC based on the selected seed\n\
trig (t) Sets the trigger for the onboard ila system to collect (burstSize) number of samples\n\
hlt (h) Halts the output to the DAC\n\
plt (p) Plots the samples that were collected by the onboard ila\n\
twave (t) Sends a triangle wave to the DAC\n\
burstSize (bs) Sets the number of samples saved by the onboard ila to be (bs) batches of samples\n\
readBurstSize (rbs) Returns the current burstSize value as well as the max\n\
scale (s) Divides the output to the dac by 2**s\n\
simple_test (st) Runs a simple memory test\n\
pwl_test (tpw) Triggers pwl sequence (testing phase)\n\
draw_pwl (dpw, T) Allows the user to draw a wave to send to the pwl (T=period of waveform in us)\n\
full_test (ft) Runs a full system test\n\n"

print("Resetting System")
dacOverlay.rst()
print("Entering Interface. Enter 'help' to display commands")

while (cmd[0] != "q"):
    cmd = removeBlanks(input("ZYNQ> ").split(' '))
    if not cmd:
        cmd = [""]
        continue
    if len(cmd) > 1: cmd[1] = eval(cmd[1])
    if (cmd[0] not in cmds):
        print("Invalid Command")
        continue

    if (cmd[0] == "help"): print(helpMsg)

    elif (cmd[0] == "choose" or cmd[0] == "c"):
        i = int(cmd[1])%len(seeds)
        seed = seeds[i]
        print(f"Selected Seed: {hex(seed)}")

    elif (cmd[0] == "burstSize" or cmd[0] == "bs"):
        if (len(cmd) != 2): print("Please select a burst size (1 arg)")
        else:
            print("Changing ila burst size")
            dacOverlay.change_burst_size(int(cmd[1]),verbose=verbose)

    elif (cmd[0] == "readBurstSize" or cmd[0] == "rbs"):
        bs,maxBs = dacOverlay.read(addr_map["ila_burst_size"]),dacOverlay.read(addr_map["max_burst_size"])
        print(f"Current samples to record: {bs*batch_size}\nMaximum samples to record: {maxBs*batch_size}")

    elif (cmd[0] == "rst" or cmd[0] == "r"):
        print("Resetting System")
        dacOverlay.rst()

    elif (cmd[0] == "random" or cmd[0] == "rs"):
        print(f"Producing random DAC samples. Seed = {hex(seed)}")
        dacOverlay.run_random_wave(seed,verbose=verbose)

    elif (cmd[0] == "trig" or cmd[0] == "t"):
        print("Setting dac_ila Trigger")
        dacOverlay.set_trigger()

    elif (cmd[0] == "hlt" or cmd[0] == "h"):
        print("Halting random DAC stream")
        dacOverlay.hlt_dac()

    elif (cmd[0] == "twave" or cmd[0] == "tw"):
        print("Initiating Triangle Wave")
        dacOverlay.run_triangle_wave(verbose=verbose)

    elif (cmd[0] == "simple_test" or cmd[0] == "st"):
        print("Beginning Simple Memory Test:")
        dacOverlay.run_mem_test_simple(verbose=verbose)

    elif (cmd[0] == "scale" or cmd[0] == "s"):
        if (len(cmd) != 2):
            print("Please provide one argument (scale factor to reduce dac output)")
            continue
        dacOverlay.set_scale(cmd[1])
        amount = 2**cmd[1] if cmd[1] <= 15 else 2**15
        print(f"Dac output divided by {amount}")

    elif (cmd[0] == "pwl_test" or cmd[0] == "tpw"):
        print("Plotting and running hardcoded pwl wave:")
        coords = [(0, 0), (2319, 3481), (6114, 5940), (12206, 7271), (12412, 10669),(15612, 8000),(16812, 8000),(20412, 10669), (20466, 7271), (25452, 7271), (28112, 5940), (32000, 0)]
        pwlf.generate_test(0,0,provided_coords=coords)
        plt.show()
        dacOverlay.prep_pwl(coords)

    elif (cmd[0] == "draw_pwl" or cmd[0] == "dpw"):
        if (len(cmd) != 2): print("Please enter a waveform period (us)")
        else:
            print("Entering MagicPen Interface:")
            coords = magic_pen.drawPath(0.99,eval(f"{cmd[1]}e-6"),make_periodic=True)
            dacOverlay.prep_pwl(coords)

    elif (cmd[0] == "full_test" or cmd[0] == "ft"):
        print("Running full system test:")
        dacOverlay.run_full_test(verbose=verbose)

    elif (cmd[0] == "plt" or cmd[0] == "p"):
        print("Plotting and clearing collected samples")
        ex_samples = dacOverlay.dac_samples.display_stats(samp_stream_pltLabels = ["Sample Generation Snapshot","Voltage (V)"], verbose = verbose)
        if (ex_samples):
            print(f"Images saved under {dacOverlay.dac_samples.name}_stats")
            ex_samples = [hex(el) for el in ex_samples]
            print(f"First 16 collected samples:{ex_samples}")
    clear_output(wait=True)
print("Exiting Interface")

CreationError: 33627 is too large a signed integer for a bitstring of length 16. The allowed range is [-32768, 32767].